In [1]:
from bs4 import BeautifulSoup
from src import Tokenizer
from time import sleep

import re
import requests

In [2]:
BASE = 'https://stackoverflow.com'
TAG = 'machine-learning'
PAGES = 0

In [3]:
def get_hrefs(page: int) -> list[str]:
    sleep(3)

    res = requests.get(f'{BASE}/questions/tagged/{TAG}?pagesize=50&page={page}&tab=Frequent')
    soup = BeautifulSoup(res.content, 'lxml')
    
    return [i.get('href') for i in soup.select('a[class=s-link]')]

In [4]:
def get_content(href: str) -> str:
    sleep(3)

    res = requests.get(f'{BASE}{href}')
    soup = BeautifulSoup(res.content, 'lxml')
    posts = soup.select('div.s-prose.js-post-body')

    content = []
    for post in posts:
        content += post.find_all('p')
    
    return ''.join(f'{i.text} ' for i in content).strip()

In [5]:

for p in range(1, PAGES+1):
    hrefs = get_hrefs(p)
    for i, h in enumerate(hrefs):
        with open(f'data/{TAG}-raw.txt', 'w', encoding='utf-8') as fp:
            print(f'Page #{p}, Post #{i} ', end='\r')
            fp.write(get_content(h))

In [6]:
with open(f'data/{TAG}-raw.txt', 'r', encoding='utf-8') as fp1:
    tokenizer = Tokenizer(fp1)
    text = tokenizer.clean_text()
    with open(f'data/{TAG}.txt', 'w', encoding='utf-8') as fp2:
        fp2.write(text)